In [2]:
import os
import cv2
import numpy as np
from random import shuffle
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam

In [3]:
TRAIN_AUTH_DIR = r'dataset\train\authentic'
TRAIN_FORG_DIR = r'dataset\train\forged'

TEST_DIR = r'dataset\test'
IMG_ROW, IMG_COL = 159, 118

In [4]:
def create_train_data():
    training_data = []
    for img in os.listdir(TRAIN_AUTH_DIR):
        if img[0] != '.' : 
            path = os.path.join(TRAIN_AUTH_DIR, img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            training_data.append([np.array(img_data), 1])
    for img in os.listdir(TRAIN_FORG_DIR):
        if img[0] != '.' : 
            path = os.path.join(TRAIN_FORG_DIR, img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            training_data.append([np.array(img_data), 0])
    
    shuffle(training_data)
    np.save(r'metadata\train_data.npy', training_data)
    return training_data

imgname = []
def create_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        if img[0] != '.' :
            path = os.path.join(TEST_DIR, img)
            imgname.append(img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            testing_data.append(np.array(img_data))

    np.save(r'metadata\test_data.npy', testing_data)
    return testing_data


train_file = r'metadata\train_data.npy'
test_file = r'metadata\test_data.npy'

if os.path.exists(train_file) and os.path.exists(test_file):
    train_data = np.load(train_file, allow_pickle=True)
    test_data = np.load(test_file, allow_pickle=True)
    for img in os.listdir(TEST_DIR):
        if img[0] != '.' :
            imgname.append(img)
    print('Found numpy data on drive. Loaded.')
    
else:
    print('Creating numpy data.')
    train_data = create_train_data()
    test_data = create_test_data()
    print('Numpy data created.')
    
print('You may proceed!')

Creating numpy data.
Numpy data created.
You may proceed!


In [5]:
X_train = np.array([x[0] for x in train_data]).reshape(-1, IMG_ROW, IMG_COL, 1)
y_train = np.array([x[1] for x in train_data])

X_test = np.array([x for x in test_data]).reshape(-1, IMG_ROW, IMG_COL, 1)

if os.path.exists(r'metadata\model.h5'):
    model = load_model(r'metadata\model.h5')
    print('Found saved model on drive. Loaded.')

else:
    print('Building model.')    
    model = Sequential()

    model.add(Conv2D(32, input_shape=(IMG_ROW, IMG_COL, 1), kernel_size=(5,5), activation='relu'))
    model.add(MaxPool2D((2)))
    model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(32, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    print('Compiling model.')
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.002), metrics=['accuracy'])
    
print('You may proceed!')

Building model.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Compiling model.
You may proceed!


In [6]:
# Fitting the model

print('Fitting Model. Please Wait.')
model.fit(X_train, y_train, batch_size=5, epochs=10, verbose=1, validation_split=0.10)

print('Model Fitting Finished.')

model.save(r'metadata\model.h5')
print('Model Saved.')
print('You may proceed!')

Fitting Model. Please Wait.
Train on 223 samples, validate on 25 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
223/223 [==============================] - 14s 65ms/sample - loss: 0.3487 - acc: 0.8117 - val_loss: 0.0112 - val_acc: 1.0000
Epoch 2/10
223/223 [==============================] - 12s 53ms/sample - loss: 0.0034 - acc: 1.0000 - val_loss: 6.1019e-04 - val_acc: 1.0000
Epoch 3/10
223/223 [==============================] - 12s 54ms/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 4.9890e-04 - val_acc: 1.0000
Epoch 4/10
223/223 [==============================] - 12s 55ms/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 4.4228e-04 - val_acc: 1.0000
Epoch 5/10
223/223 [==============================] - 12s 53ms/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 4.0783e-04 - val_acc: 1.0000
Epoch 6/10
223/223 [==============================] - 12s 52ms/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 3.8055e-04 - val_acc: 1.0000
Epoch 7/10
223/223 [=======================

In [7]:
output = model.predict(X_test)

print('Authenticity predicted : ')
for i in range(len(output)):
    print(imgname[i], ": %.3f %%" % (output[i]*100))

Authenticity predicted : 
img 2019-12-02 11-34-16.png : 99.962 %
img 2019-12-02 11-34-50.png : 6.388 %
img 2019-12-02 11-35-14.png : 14.752 %
img 2019-12-02 11-35-30.png : 3.615 %
